In [12]:
import chromadb
client=chromadb.Client()
collection=client.create_collection(name="my_collection")

UniqueConstraintError: Collection my_collection already exists

In [13]:
collection.add(
    documents=[
        "new york",
        "new delhi"
    ],
    ids=['id1','id2']
)

C:\Users\Mithun Pranava\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|█| 79.3M/79.3M [01:34<00:00, 882ki


In [14]:
all_docs=collection.get(ids=["id1"])
all_docs

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['new york'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [15]:
results=collection.query(
    query_texts=["chole bhature"],
    n_results=2
)
results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['new delhi', 'new york']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.7241712808609009, 2.044581174850464]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/?jobSearch=true&jsOffset=0&jsSort=posting_start_date&jsLanguage=en&error=jobPost")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Nike Careers Home

Section 1: Move the World. Welcome to Nike Careers.Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsMove the WorldWelcome to Nike CareersSearch JobsScrollSearch JobsFin

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-45280?from=job%20search%20funnel")
page_data=loader.load().pop().page_content
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)
prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from careers page of a website.
    your job is to extract jobs postings and return them in the json format containing the 
    following keys: 'role','experience','skills' and 'description'.
    only return the valid json.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
  {
    "role": "Analyst, Finance GC",
    "experience": "At least 5 years’ experience of audit or finance analyst position in MNC, Media or Creative agency working experience is a plus",
    "skills": "Proficiency with office software such as Microsoft Excel, Outlook, Word and PowerPoint, Power BI and tableau skill is preferred, Excellent English oral and written skills",
    "description": "This is a commercial FP&A role and key responsibility is to support Icon. Shanghai Studio Financial planning and analysis for Business team. Support monthly actual and forecast spending collection, validation, and variance analysis. Support system maintenance and development to facilitate Icon financial process. Support annual budgeting and 3 years strategic plan development. Prepare financial related review reports and deliverables. Cross function communication with Icon. Studio team and other finance team. Other ad hoc tasks"
  }
]


In [14]:
type(res.content)

str

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
print(json_res)

[{'role': 'Analyst, Finance GC', 'experience': 'At least 5 years’ experience of audit or finance analyst position in MNC, Media or Creative agency working experience is a plus', 'skills': 'Proficiency with office software such as Microsoft Excel, Outlook, Word and PowerPoint, Power BI and tableau skill is preferred, Excellent English oral and written skills', 'description': 'This is a commercial FP&A role and key responsibility is to support Icon. Shanghai Studio Financial planning and analysis for Business team. Support monthly actual and forecast spending collection, validation, and variance analysis. Support system maintenance and development to facilitate Icon financial process. Support annual budgeting and 3 years strategic plan development. Prepare financial related review reports and deliverables. Cross function communication with Icon. Studio team and other finance team. Other ad hoc tasks'}]


In [17]:
type(json_res)

list

In [32]:
job=json_res

In [33]:
job

[{'role': 'Analyst, Finance GC',
  'experience': 'At least 5 years’ experience of audit or finance analyst position in MNC, Media or Creative agency working experience is a plus',
  'skills': 'Proficiency with office software such as Microsoft Excel, Outlook, Word and PowerPoint, Power BI and tableau skill is preferred, Excellent English oral and written skills',
  'description': 'This is a commercial FP&A role and key responsibility is to support Icon. Shanghai Studio Financial planning and analysis for Business team. Support monthly actual and forecast spending collection, validation, and variance analysis. Support system maintenance and development to facilitate Icon financial process. Support annual budgeting and 3 years strategic plan development. Prepare financial related review reports and deliverables. Cross function communication with Icon. Studio team and other finance team. Other ad hoc tasks'}]

In [20]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [28]:
import uuid
import chromadb

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _ , row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links":row["Links"]},
            ids=[str(uuid.uuid4())]
        )
links=collection.query(query_texts=["Experience in python","Expertise i React"],n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [34]:
prompt_email = PromptTemplate.from_template(
        
        """
        
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Efficient Financial Planning and Analysis with AtliQ

Dear Hiring Manager,

I came across the Analyst, Finance GC role at your esteemed organization and was impressed by the emphasis on leveraging technology to drive financial planning and analysis. As a Business Development Executive at AtliQ, I'd like to introduce our expertise in empowering businesses like yours with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team at AtliQ has extensive experience in developing and implementing automated tools that streamline financial processes, including financial planning, analysis, and reporting. We've successfully empowered numerous enterprises with our solutions, and I'd like to highlight a few relevant examples from our portfolio:

* Our machine learning and Python expertise have enabled clients to automate complex financial tasks, such as forecasting and variance analysis. You can explore our capab